## Zadanie 1

In [11]:
from dask.distributed import Client

client = Client(n_workers=4, memory_limit='2GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 7.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:64101,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 7.45 GiB
Comm: tcp://127.0.0.1:64113,Total threads: 2
Dashboard: http://127.0.0.1:64116/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:64104,


In [24]:
import dask
import dask.array as da
from dask_ml.datasets import make_classification
from dask_ml.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from dask_ml.wrappers import Incremental

n, d = 6000000, 400
X, y = make_classification(n_samples=n, n_features=d,
                           chunks=n // 420, flip_y=0.2)

X_train, X_test, y_train, y_test = train_test_split(X, y)

classes = da.unique(y_train).compute()

est = SGDClassifier(loss='log_loss', penalty='l2', tol=1e-3)
inc = Incremental(est, scoring='accuracy')

inc.fit(X_train, y_train, classes=classes)
score = inc.score(X_test, y_test)
print(f"Dokładność modelu: {score}")


/Users/konradzieba/anaconda3/lib/python3.11/site-packages/dask/base.py:1541: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


Dokładność modelu: 0.6498258942703388


## Zadanie 2

In [26]:
import joblib
joblib.dump(inc, 'incremental_model.joblib')

['incremental_model.joblib']

In [28]:
from sklearn.metrics import confusion_matrix, classification_report

loaded_model = joblib.load('incremental_model.joblib')

predictions = loaded_model.predict(X_test)

y_test_np = y_test.compute()
predictions_np = predictions.compute()

conf_matrix = confusion_matrix(y_test_np, predictions_np)
print("Macierz klasyfikacji:")
print(conf_matrix)

Macierz klasyfikacji:
[[194196 105697]
 [104481 195836]]


In [30]:
class_report = classification_report(y_test_np, predictions_np)
print("Raport klasyfikacji:")
print(class_report)

score = loaded_model.score(X_test, y_test)
print(f"Dokładność modelu po wczytaniu: {score}")

Raport klasyfikacji:
              precision    recall  f1-score   support

           0       0.65      0.65      0.65    299893
           1       0.65      0.65      0.65    300317

    accuracy                           0.65    600210
   macro avg       0.65      0.65      0.65    600210
weighted avg       0.65      0.65      0.65    600210

Dokładność modelu po wczytaniu: 0.6498258942703388


In [21]:
from dask.distributed import Client
from dask_ml.model_selection import IncrementalSearchCV
from dask_ml.datasets import make_classification
from sklearn.linear_model import SGDClassifier
import numpy as np

X, y = make_classification(n_samples=5000000, n_features=20,
                           chunks=100000, random_state=56)

model = SGDClassifier(tol=1e-3, penalty='elasticnet', random_state=0)

params = {
    'alpha': np.logspace(-2, 1, num=1000),
    'l1_ratio': np.linspace(0, 1, num=1000),
    'average': [True, False]
}

search = IncrementalSearchCV(
    model,
    params,
    n_initial_parameters=1000,
    patience=20,
    max_iter=100,
    random_state=0
)

search.fit(X, y, classes=[0, 1])

print("Najlepsze parametry:", search.best_params_)
print("Najlepszy wynik (accuracy):", search.best_score_)

/Users/konradzieba/anaconda3/lib/python3.11/site-packages/dask/base.py:1541: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(
/Users/konradzieba/anaconda3/lib/python3.11/site-packages/dask_ml/model_selection/_incremental.py:1039: FutureWarning: decay_rate has been deprecated since Dask-ML v1.4.0.

    * Use InverseDecaySearchCV to use `decay_rate`
    * Specify decay_rate=None


  warn(


Najlepsze parametry: {'l1_ratio': 0.002002002002002002, 'average': False, 'alpha': 0.5711586478126432}
Najlepszy wynik (accuracy): 0.67707


In [ ]:
client.close()